In [2]:
# 最適化の近似がどの程度できているのか？

In [3]:
import sys
import os
import glob
import random
import torch
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import  open_json
import copy
from utils.util import calc_increase_c
from utils.versatility import calc_increase_v
from utils.util import calc_cw_score
from itertools import combinations

In [4]:
filepaths = glob.glob(
    f"C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
)

In [5]:
id_to_vector_versatility = open_json('C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_versatility.json')
id_to_vector_compatibility = open_json('C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_compatibility.json')

In [6]:
from utils.util import filter_basic_items, get_category

items = {"tops": [], "bottoms": [], "shoes": []}
for fp in filepaths:
    json_dict = open_json(fp)
    if json_dict == None:
        continue
    for item in filter_basic_items(json_dict["items"]):
        category = get_category(item)
        if category == None:
            continue
        items[category].append(str(item['itemId']))

In [7]:
categories = ['tops', 'bottoms', 'shoes']
# 枚数
T = 3

In [8]:
import importlib
import utils
importlib.reload(utils.util)
from utils.util import calc_cw_score
from utils.util import calc_increase_c

In [12]:
for i in range(100):
    dataset = {
        'tops':random.sample(items["tops"], 10),
        'bottoms':random.sample(items["bottoms"], 10),
        'shoes':random.sample(items["shoes"], 10),
    }

    # cwの初期化
    cw = {
        'tops':random.sample(dataset["tops"], T),
        'bottoms':random.sample(dataset["bottoms"], T),
        'shoes':random.sample(dataset["shoes"], T),
    }
    eps = 10e-3
    increase_score = eps + 1
    pre_cw_score = 0
    while increase_score > eps:
        cw_score_confirm = 0
        for cat in categories:
            cw[cat] = []
            ver_set = set()
            com_score = 0
            for _ in range(T):
                add_item = None
                max_increase_score = 10e-5
                # layer = copy.copy(cw[cat])
                for item in dataset[cat]:
                    if item in cw[cat]:
                        continue
                    # print(item in cw[cat], item, cw[cat])
                    increase_c = calc_increase_c(cw, id_to_vector_compatibility, item, cat)
                    add_clusters = calc_increase_v(torch.tensor(id_to_vector_versatility[item]))
                    increase_v = len(add_clusters.union(ver_set)) - len(ver_set)
                    print(increase_c, increase_v)
                    increase = increase_c + increase_v
                    if max_increase_score < increase:
                        max_increase_score = increase
                        add_item = item
                
                com_score += calc_increase_c(cw, id_to_vector_compatibility, add_item, cat)
                ver_set = ver_set.union(calc_increase_v(torch.tensor(id_to_vector_versatility[add_item])))
                # print(com_score)
                cw[cat].append(add_item)
            # cw_score_confirm += com_score + len(ver_set)
            cw_score_confirm += com_score
        score = sum(calc_cw_score(cw, id_to_vector_versatility, id_to_vector_compatibility))
        increase_score = score - pre_cw_score
        pre_cw_score = score

    c, v = calc_cw_score(cw, id_to_vector_versatility, id_to_vector_compatibility)
    submoduler = score
    # print(f'{i} fin submoduler', c, v)
    continue
    score = 0
    i = 0
    for tops in list(combinations(dataset['tops'], 3)):
        for bottoms in list(combinations(dataset['bottoms'], 3)):
            for shoes in list(combinations(dataset['shoes'], 3)):
                i += 1
                if i % 10000 == 0:
                    print(f'\r{i * 100 / pow(120, 3)}%', end='')
                cw = {
                    'tops': tops,
                    'bottoms': bottoms,
                    'shoes': shoes
                }
                score = max(score, sum(calc_cw_score(cw, id_to_vector_versatility, id_to_vector_compatibility)))
    with open('C:/Users/yuuta/Documents/fashion/experiments/optimization/result/optim.txt', 'w') as f:
        f.write(f'{submoduler} {score}\n')

tensor(186.1922) 210
tensor(210.1056) 210
tensor(347.8545) 210
tensor(399.6275) 210
tensor(169.3526) 210
tensor(148.4176) 210
tensor(148.4176) 210
tensor(148.4176) 210
tensor(148.4176) 210
tensor(160.3804) 210
tensor(186.1922) 4
tensor(210.1056) 4
tensor(347.8545) 6
tensor(169.3526) 92
tensor(148.4176) 0
tensor(148.4176) 3
tensor(148.4176) 24
tensor(148.4176) 2
tensor(160.3804) 25
tensor(186.1922) 4
tensor(210.1056) 2
tensor(169.3526) 90
tensor(148.4176) 0
tensor(148.4176) 1
tensor(148.4176) 22
tensor(148.4176) 0
tensor(160.3804) 25
tensor(221.5404) 210
tensor(200.9395) 210
tensor(150.5463) 210
tensor(150.9237) 210
tensor(169.3526) 210
tensor(150.5463) 210
tensor(226.9550) 210
tensor(179.6979) 210
tensor(150.5463) 210
tensor(196.1674) 210
tensor(221.5404) 19
tensor(200.9395) 3
tensor(150.5463) 33
tensor(150.9237) 8
tensor(169.3526) 5
tensor(150.5463) 5
tensor(179.6979) 7
tensor(150.5463) 116
tensor(196.1674) 19
tensor(221.5404) 6
tensor(200.9395) 1
tensor(150.5463) 0
tensor(150.9237) 0

In [10]:
# compatibilityは、最小化しなきゃいけない値